 FIFO 队列:先添加的任务会先被提取。
 LIFO 队列:最近添加的条目会先被提取 (类似于一个栈)。
 优先级队列: 条目将保持已排序状态 (使用 heapq 模块) 并且值最小的条目会先被提取。

 SimpleQueue能让你在多线程编程中优雅安全地传递数据。

 >如何理解这句话？ 状态检查的局限性：.empty()和 .qsize()的返回值在多线程环境下是“瞬间”状态

 用一个生动的比喻来解释多线程队列状态检查的局限性，希望能让你一下子抓住核心要点。想象一下这个场景：

你正在超市的一个收银台前，想知道是否轮到自己结账。队列的状态（有无人排队）就像 .empty()和 .qsize()返回的结果。

你探头看了一眼队伍，发现前面没人（queue.empty()返回 True）。但就在你收回目光，准备走上前的那一刹那，一位身手敏捷的顾客“嗖”地一下从旁边闪入，排到了你前面。对你而言，刚才“队伍为空”的判断在瞬间就过时了。

同样，你可能看到队伍有3个人（queue.qsize()返回 3），但当你还在数的时候，可能同时发生两件事：一个新顾客排到了队尾（人数变为4），而收银员又服务完了一个顾客（人数又变回3）。你看到的“3”只是一个瞬间的快照，实际的队伍长度可能在你看完的下一秒就变了。


🔍 为什么“看一眼”的结果不可靠？
在编程中，.empty()和 .qsize()就像这“探头看一眼”。它们执行的速度极快，但在多线程环境下，当你得到返回值后、准备基于这个返回值进行下一步操作（如取物品）前，CPU完全可能切换到另一个正在操作同一个队列的线程上去工作一会儿。等再切回你的线程时，队列的状态很可能已经变了。

因此，依赖于这些“瞬间状态”的返回值来做后续逻辑判断，本质上是不安全的。你不能相信“如果队列不为空，我就去取”这样的逻辑，因为判断“不为空”之后，队列可能立刻被其他线程清空，导致你的 get()操作阻塞或需要处理异常。

✅ 更可靠的做法是什么？
最可靠的方式，是放弃“先检查、后行动”的思路，转而采取一种 “直接行动，并准备好等待或处理异常”​ 的策略。这就像在收银台，最稳妥的方式不是不断张望，而是直接排到队尾（如果有人在排队）或者直接开始结账（如果刚好没人），并接受可能需要短暂等待的事实。

在 SimpleQueue中，这就意味着：

对于消费者线程，直接调用 .get()​ 方法。如果队列为空，该方法会自动阻塞你的线程，直到有数据可用。你不需要事先检查队列是否为空，队列内部机制会妥善处理同步问题。

如果你不希望线程无限期阻塞，可以给 .get()方法设置 timeout参数，或者在特定情况下使用非阻塞的 .get_nowait()方法（但需准备好捕获 queue.Empty异常）。

这种方法将状态检查和获取数据合并成了一个原子性的操作，从而避免了基于不可靠的中间状态做决策的风险
。

希望这个比喻能帮助你直观地理解多线程环境下队列状态检查的局限性。关键在于理解并发环境下“瞬间状态”的不可靠性，并转而依赖队列内部提供的、更安全的同步机制。


小结：
SimpleQueue、Queue、LifoQueue和 PriorityQueue也同样不建议在多线程环境下依赖 .empty()和 .qsize()等方法进行关键的流程控制决策


In [ ]:
from queue import SimpleQueue
import threading
import time

# 创建一个全局的 SimpleQueue 实例
# 无界队列，不限制队列长度
# 如果生产者的速度持续远快于消费者，队列占用的内存会不断增长，存在耗尽内存的风险。
message_queue = SimpleQueue()
def producer():
    """生产者线程函数，负责向队列放入数据"""
    for i in range(5):
        message = f"消息-{i}"
        message_queue.put(message)  # 将消息放入队列
        print(f"[生产者] 放入了: {message}")
        time.sleep(0.5)  # 模拟生产过程耗时
    print("[生产者] 所有任务已发送完毕！")

def consumer():
    """消费者线程函数，负责从队列取出并处理数据"""
    while True:
        # 从队列获取消息。如果没有消息，会在此阻塞等待。
        message = message_queue.get()
        print(f"[消费者] 处理了: {message}")
        # 注意：SimpleQueue 没有 task_done() 和 join() 方法。
        # 这里我们用一个简单的条件来判断是否退出：当收到特定的结束信号时
        if message == "消息-4":
            print("[消费者] 收到结束信号，准备退出。")
            break

# 创建线程对象
producer_thread = threading.Thread(target=producer)
consumer_thread = threading.Thread(target=consumer)
# 启动线程
producer_thread.start()
consumer_thread.start()

# 等待两个线程执行完毕
producer_thread.join()
consumer_thread.join()
print("主程序执行结束。")


[生产者] 放入了: 消息-0
[消费者] 处理了: 消息-0
[生产者] 放入了: 消息-1[消费者] 处理了: 消息-1

[生产者] 放入了: 消息-2[消费者] 处理了: 消息-2

[生产者] 放入了: 消息-3[消费者] 处理了: 消息-3

[生产者] 放入了: 消息-4[消费者] 处理了: 消息-4
[消费者] 收到结束信号，准备退出。

[生产者] 所有任务已发送完毕！
主程序执行结束。


In [ ]:
import threading
import time
import queue

# 创建一个队列，并放入一个数据
my_queue = queue.Queue()
my_queue.put("任务A")


def careless_consumer():
    """一个粗心的消费者线程，它相信‘看一眼’的结果"""
    time.sleep(0.1)  # 稍微延迟一下，让另一个线程先运行
    print(f"[粗心消费者] 我看了一眼队列，它是空的吗？{my_queue.empty()}")

    # ⚠️ 危险操作：基于“看一眼”的结果做决策
    if not my_queue.empty():
        # 在打印完上面那句话后，到执行下面这行代码前，队列状态可能已经变了！
        time.sleep(1)  # 模拟线程在做其他事情（比如找碗），这给了其他线程可乘之机
        try:
            item = my_queue.get(timeout=1)  # 尝试获取，但此时可能已空
            print(f"[粗心消费者] 成功获取到: {item}")
        except queue.Empty:
            print("[粗心消费者] 错误！刚才明明不是空的，但现在get不到东西了！")
    else:
        print("[粗心消费者] 队列是空的，我什么都不做。")


def reliable_consumer():
    """ 可靠消费者 """
    while True:
        try:
            # 直接尝试获取，不先检查
            item = my_queue.get(timeout=5)  # 等待5秒，如果还为空则抛出Empty异常
            print(f"[可靠消费者] 处理任务: {item}")
            my_queue.task_done()  # 标记任务处理完成
        except queue.Empty:
            print("[可靠消费者] 一段时间内没有新任务，退出。")
            break


def fast_consumer():
    """一个手脚麻利的消费者线程，它抢在粗心线程之前行动"""
    time.sleep(0.2)  # 它比粗心线程晚0.1秒启动，但动作更快
    if not my_queue.empty():
        item = my_queue.get()  # 迅速取走任务
        print(f"[敏捷消费者] 嗖~ 我抢先取走了: {item}")



# 启动线程
t1 = threading.Thread(target=careless_consumer)
t2 = threading.Thread(target=fast_consumer)

t1.start()
t2.start()

t1.join()
t2.join()

print("程序结束。")

"""
⚡ 代码解释
[粗心消费者]​ 线程首先检查队列，发现不为空（False）。
但在它执行 time.sleep(1)模拟耗时操作时，CPU 切换到了 [敏捷消费者]​ 线程。
[敏捷消费者]​ 线程迅速调用 .get()取走了队列中唯一的任务。
当 CPU 再次切换回 [粗心消费者]​ 线程时，它试图执行 .get()，但队列已经空了，于是触发 queue.Empty异常。
这个例子完美演示了：在一个线程中判断队列状态后，在它采取行动前，这个状态完全可能被其他线程改变。


这就好比，最稳妥的方式不是反复张望队伍有没有人，而是直接去排队。队列这个数据结构本身就为你处理好了所有的同步问题。

希望这个例子和解释能让你彻底明白这个问题！这是一个非常重要的多线程编程概念。理解了它，你就能避免一大类难以调试的并发bug。
"""


[粗心消费者] 我看了一眼队列，它是空的吗？False
[敏捷消费者] 嗖~ 我抢先取走了: 任务A
[粗心消费者] 错误！刚才明明不是空的，但现在get不到东西了！
程序结束。


In [ ]:
# 经典的消费者模式
import queue
import threading
import time

simple_queue = queue.SimpleQueue()

def consumer(simple_queue: SimpleQueue):
    while True:
        try:
            item = simple_queue.get(timeout=2)
            if item is None:
                print("[消费者] 获取到None，退出。")
                break
            print(f"[消费者] 处理任务: {item}")
            time.sleep(0.2)  # 模拟处理耗时
        except queue.Empty:
            print("[消费者] 一段时间内没有新任务，退出。")
            break

def producer(simple_queue:queue.SimpleQueue):
    """生产者"""
    try:
        for i in range(5):
            item = f"任务{i}"
            print(f"[生产者] 添加任务: {item}")
            simple_queue.put(item,timeout=1)
            time.sleep(0.2)
    except queue.Full:
        print("[生产者] 队列已满，无法添加更多任务。")

prod_task = threading.Thread(target=producer, args=(simple_queue,))
cons_task = threading.Thread(target=consumer, args=(simple_queue,))

prod_task.start()
cons_task.start()
prod_task.join()
cons_task.join()
print("程序结束。")


[生产者] 添加任务: 任务0
[消费者] 处理任务: 任务0
[生产者] 添加任务: 任务1
[消费者] 处理任务: 任务1
[生产者] 添加任务: 任务2
[消费者] 处理任务: 任务2
[生产者] 添加任务: 任务3
[消费者] 处理任务: 任务3
[生产者] 添加任务: 任务4
[消费者] 处理任务: 任务4
[消费者] 一段时间内没有新任务，退出。
程序结束。


In [ ]:


def producer():
    """生产者线程"""
    for i in range(10):
        item = f"任务{i}"
        my_queue.put(item)
        print(f"[生产者] 添加了任务: {item}")
        time.sleep(0.5)  # 模拟耗时操作
